# Extract Lane Info from Images

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob

# Sobel Thresholding

In [3]:
#Applying Gradient based thresholding
def sobel_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = img[:,:,1]
    
    kernelSize=3
    #pad=np.int_((kernelSize-1)/2)
    #Pad image
    #gray = cv2.copyMakeBorder( gray, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(255,255,255))
    xImg=abs_sobel_thresh(gray, 'x', kernelSize, (30, 200))
    yImg=abs_sobel_thresh(gray, 'y', kernelSize, (30, 200))
    magImg=mag_thresh(gray, kernelSize, (30, 200))
    dirImg=dir_threshold(gray, kernelSize, (0.7, 1.3))
    
    sobelThresh=np.zeros_like(gray).astype(np.uint8)
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1)) | ((xImg == 1) & (yImg == 1))] = 1
    #sobelThresh=sobelThresh[pad:-pad,pad:-pad]
    
    #Fine-tuning thresholding condition 
    debug=0
    if (debug==1):
        test_sobel_thresh(xImg,yImg,magImg,dirImg)
    
    return sobelThresh

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return the binary image
    return binary_output

def test_sobel_thresh(xImg,yImg,magImg,dirImg):
    sobelThresh=np.zeros_like(xImg).astype(np.uint8)
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X-dirn')
    plt.show()
    
    sobelThresh[((yImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('Y-dirn')
    
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1))|((yImg == 1) & (magImg == 1) & (dirImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X or Y-dirn')
    
    sobelThresh[((xImg == 1) & (magImg == 1) & (dirImg == 1)) | ((xImg == 1) & (yImg == 1))] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('X or X with Y')

# HLS Thresholding

In [4]:
#Apply color based Thresholding
def aug_HLS_thresh(sobelThresh, img):
    # Adaptive Histogram Equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eq1=clahe.apply(img[:,:,0])
    eq2=clahe.apply(img[:,:,1])
    eq3=clahe.apply(img[:,:,2])
    eqHistImg=cv2.merge((eq1,eq2,eq3))
    #eqHistImg=img
    
    #HLS Thresholding
    hlsThresh=hls_thresh(eqHistImg, (0,50), (150, 255))
    #Combine Sobel and HLS
    final=combine_sobel_hls_thresh(sobelThresh, hlsThresh)
    
    #plot hls and aug images
    debug=0
    if (debug==1):
        test_aug_thresh(eqHistImg,sobelThresh,hlsThresh,final)
    
    return final

def hls_thresh(img, Hthresh=(0,100), Sthresh=(80, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    binary = np.zeros_like(s_channel)
    binary[(h_channel > Hthresh[0]) & (h_channel <= Hthresh[1]) & 
           (s_channel > Sthresh[0]) & (s_channel <= Sthresh[1])] = 1
    return binary
    
def combine_sobel_hls_thresh(sobel, hls):
    result = np.zeros_like(hls).astype(np.uint8)
    result[((sobel == 1) | (hls == 1))] = 1
    return result

def test_aug_thresh(img,sobelThresh,hlsThresh,final):
    hls=cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    
    plt.figure(figsize = (16,16))
    plt.imshow(h_channel,cmap='gray')
    plt.title('H')
    
    
    binary = np.zeros_like(s_channel)
    binary[(h_channel > 0) & (h_channel <= 50)] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(binary,cmap='gray')
    plt.title('H Threshold')
    
    plt.figure(figsize = (16,16))
    plt.imshow(s_channel,cmap='gray')
    plt.title('S')
    
    binary = np.zeros_like(s_channel)
    binary[(s_channel > 150) & (s_channel <= 255)] = 1
    plt.figure(figsize = (16,16))
    plt.imshow(binary,cmap='gray')
    plt.title('S Threshold')
    
    plt.figure(figsize = (16,16))
    plt.imshow(sobelThresh,cmap='gray')
    plt.title('Sobel')
    plt.show()
    
    plt.figure(figsize = (16,16))
    plt.imshow(hlsThresh,cmap='gray')
    plt.title('HLS')
    plt.show()
    
    plt.figure(figsize = (16,16))
    plt.imshow(final,cmap='gray')
    plt.title('Final')
    plt.show()